# Labelling of cables and suspended streetlights

We label cables by locating 'linear point' clusters above ground.

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import numpy as np
import logging

import upcp.utils.log_utils as log_utils
import upcp.utils.ahn_utils as ahn_utils
import upcp.utils.bgt_utils as bgt_utils
import upcp.fusion as fusion
from upcp.labels import Labels
from upcp.pipeline import Pipeline

# INFO messages will be printed to console.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

## Cable fuser based on point neighbourhood.
First, download and parse the required BGT and BAG data in notebook [1. Generate reference data](1.%20Generate%20reference%20data.ipynb).

In [ ]:
# The already labelled ground and building points must be removed.
exclude_labels = (Labels.GROUND, Labels.ROAD, Labels.BUILDING)

In [ ]:
# Select the file to process.
tilecode = '2386_9702'
# tilecode = '2397_9705'

in_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'
out_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'

# Data folder for the fusers.
ahn_data_folder = '../datasets/ahn/'
pc_data_folder = '../datasets/pointcloud/'
bag_building_file = '../datasets/bag/bag_buildings_demo.csv'
bgt_tramtrack_file = '../datasets/bgt/bgt_tramtracks_demo.csv'

# We need elevation data to determine object height above ground.
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

# Create reader for BAG outlines.
bag_building_reader = bgt_utils.BGTPolyReader(bgt_file=bag_building_file)

# Create reader for BGT road part polygons.
bgt_tramtrack_reader = bgt_utils.BGTLineReader(bgt_file=bgt_tramtrack_file)

For Cable fuser parameters, see [cable_fuser.py](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/blob/main/src/upcp/fusion/cable_fuser.py)

In [ ]:
# Car fuser using a clustering algorithm and BGT road data.
cable_fuser = fusion.CableFuser(Labels.CABLE, Labels.CABLE, Labels.TRAM_CABLE, Labels.ARMATUUR, ahn_reader, bag_reader=bag_building_reader, bgt_tram_reader=bgt_tramtrack_reader)

## Process point clouds and save the results
The pipeline can now be used to process point clouds. Labels will be written to the LAS file as a new extended field called label.

In [ ]:
# Set-up the order in how we want to label.
process_sequence = (cable_fuser,)
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

## Process a single file

In [ ]:
# Process the file.
pipeline.process_file(in_file, out_file=out_file)